In [5]:
import pandas as pd
import numpy as np
from shapely.geometry import Point
import geopandas as gpd
import requests
import argparse
import sys
from fuzzywuzzy import process
# -*- coding: utf-8 -*-


In [6]:
def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c
def distance_meters(mercator_start, mercator_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    return mercator_start.distance(mercator_finish)

In [7]:
def monuments():
    response = requests.get('https://datos.madrid.es/egob/catalogo/208844-0-monumentos-edificios.json')
    resultado = response.json()
    monuments = pd.json_normalize(resultado['@graph'])
    monuments["Type of place"]= "Edificios de carácter monumental"
    monuments = monuments.rename(columns={"title": "Place of interest", "address.street-address": "Place address", "location.longitude": "Long_start", "location.latitude": "Lat_start"}, errors="raise")
    monuments_clean = monuments[["Place of interest","Type of place","Place address","Long_start","Lat_start"]]
    monuments_clean['Lat_start'] = pd.to_numeric(monuments_clean['Lat_start'],errors = 'coerce')
    monuments_clean['Long_start'] = pd.to_numeric(monuments_clean['Long_start'],errors = 'coerce')
    monuments_clean = monuments_clean.fillna(0)
    return monuments_clean

def stations():
    stations = pd.read_json("../data/bicimad_stations.json", orient='records')
    geometry_coordinates = stations["geometry_coordinates"].str.split(expand=True)
    geometry_coordinates.columns = ['LONGITUD', 'LATITUD']
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace("[","")
    geometry_coordinates['LONGITUD'] = geometry_coordinates['LONGITUD'].str.replace(",","")
    geometry_coordinates['LATITUD'] = geometry_coordinates['LATITUD'].str.replace("]","")
    stations = pd.concat([stations, geometry_coordinates], axis=1)
    stations_clean = stations.rename(columns={"name": "BiciMAD station", "address": "Station location", "LONGITUD": "Long_finish", "LATITUD": "Lat_finish"}, errors="raise")
    stations_clean = stations_clean[["BiciMAD station","Station location","Long_finish","Lat_finish"]]
    stations_clean['Lat_finish'] = pd.to_numeric(stations_clean['Lat_finish'],errors = 'coerce')
    stations_clean['Long_finish'] = pd.to_numeric(stations_clean['Long_finish'],errors = 'coerce')
    stations_clean = stations_clean.fillna(0)
    return stations_clean


In [8]:
def mercator_1():
    monuments_clean["Mercator_start"] = monuments_clean.apply(lambda x: to_mercator(x['Lat_start'],x['Long_start']),axis=1)
    return monuments_clean
def mercator_2():
    stations_clean["Mercator_finish"] = stations_clean.apply(lambda x: to_mercator(x['Lat_finish'],x['Long_finish']),axis=1)
    return stations_clean

In [9]:
def merge():
    df_sumary = pd.merge(monuments_clean,stations_clean, how="cross")
    return df_sumary

In [10]:
def apply_distance_meters():
    df_sumary["Distance"] = df_sumary.apply(lambda x: distance_meters(x['Mercator_start'],x['Mercator_finish']),axis=1)
    return df_sumary


In [47]:
def minimum():
    ok= list(monuments_clean["Place of interest"].unique())
    usuario = input('Pon el lugar de interés: ')
    aprox=process.extractOne(usuario, ok, score_cutoff=80)
    df_filter = df_sumary[df_sumary["Place of interest"] == aprox[0]]
    pre_minimum = df_filter[df_filter['Distance'] == df_filter['Distance'].min()]
    minimum = pre_minimum[["Place of interest","Type of place","Place address","BiciMAD station","Station location","Distance"]]
    return minimum


In [12]:
#def all_places():
    #df_minimum = pd.DataFrame(columns=["Place of interest","Type of place","Place address","BiciMAD station","Station location"])
    #for place in monuments_clean["Place of interest"]:
        #df_filter2 = df_sumary[df_sumary["Place of interest"] == place]
        #df_places =  df_filter2[df_filter2['Distance'] == df_filter2['Distance'].min()]
        #df_places_clear = df_places[["Place of interest","Type of place","Place address","BiciMAD station","Station location"]]
        #df_minimum = df_minimum.append(df_places_clear)
    #return df_minimum

    

In [13]:
def all_places():  
    return df_sumary.sort_values(by = "Distance", ascending = True).groupby('Place of interest')['Type of place','Place address','BiciMAD station', 'Station location','Distance'].nth(0).drop(["Distance"], axis = "columns") 



In [14]:
stations_clean = stations()
monuments_clean = monuments()
monuments_clean = mercator_1()
stations_clean = mercator_2()
df_sumary = merge()
df_sumary = apply_distance_meters()
all_places()

/Users/albertofernandez/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/Users/albertofernandez/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/Users/albertofernandez/miniconda3/envs/proy-env/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Type of place,Place address,BiciMAD station,Station location
Place of interest,,,,
Almacenes Rodríguez,Edificios de carácter monumental,Calle CABALLERO DE GRACIA 3,Plaza de Pedro Zerolo,Plaza de Pedro Zerolo nº 1
Antigua Fábrica Martini &amp; Rossi,Edificios de carácter monumental,AVENIDA ARAGON 328,Gutierre de Cetina,Calle Gutierre de Cetina nº 77
Ateneo de Madrid,Edificios de carácter monumental,Calle PRADO 21,Plaza de Santa Ana,Plaza de Santa Ana nº 10
Banco Bilbao Vizcaya Argentaria,Edificios de carácter monumental,Calle ALCALÁ 16,Sevilla,Calle Alcalá nº 27
Banco Matritense,Edificios de carácter monumental,Calle GRAN VIA 22,Plaza de Pedro Zerolo,Plaza de Pedro Zerolo nº 1
...,...,...,...,...
Viviendas y oficinas para Seguros La Estrella,Edificios de carácter monumental,Calle CABALLERO DE GRACIA 15,Sevilla,Calle Alcalá nº 27
Viviendas y oficinas para el vizconde de Escoriaza,Edificios de carácter monumental,Calle GRAN VIA 22B,Plaza de Pedro Zerolo,Plaza de Pedro Zerolo nº 1
Viviendas y oficinas para la Constructora Calpense,Edificios de carácter monumental,Calle CHINCHILLA,Tres Cruces,Calle Tres Cruces nº 7


In [48]:
minimum()

Pon el lugar de interés: Zo Acarium d Madriz


,Place of interest,Type of place,Place address,BiciMAD station,Station location,Distance
55930,Zoo Aquarium de Madrid,Edificios de carácter monumental,COMPLEJO ZOOLOGICO DE LA CASA CAMPO,Caramuel,Calle Caramuel nº 51,3673.686457
